In [1]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from io import BytesIO
from urllib.parse import urljoin, urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from PIL import Image
import os
import random
import re
import pandas as pd
import hashlib
from deep_translator import GoogleTranslator
from datetime import datetime
import urllib.parse
from json import dumps, loads
from shutil import copy2

In [2]:
def translate(text):
    return GoogleTranslator(source='ko', target='en').translate(text=text).replace(',', '').replace('\n', '')

def create_directories(base_url, categories, label='images'):
    # create the following dir struct; outputs > base website > categories
    base_dir = os.path.join('output', label, urlparse(base_url).netloc)
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    for category, _ in categories:
        category_dir = os.path.join(base_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)

    return base_dir

In [3]:
url = 'https://www.infobae.com/entretenimiento/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')

In [8]:
main_content = soup.find('article', class_='article').find('div', class_='feed-list-wrapper')

for a_tag in main_content.find_all('a'):
    print(a_tag['href'])

/entretenimiento/2024/09/21/un-corazon-roto-y-solo-10-minutos-asi-nacio-dreams-el-exito-de-stevie-nicks-para-fleetwood-mac/
/entretenimiento/2024/09/21/5-bandas-que-cambiaron-de-cantante-y-no-tuvieron-exito/
/entretenimiento/2024/09/20/ridley-scott-ya-planea-una-tercera-entrega-de-gladiador/
/noticias/2024/09/19/maraton-de-k-dramas-no-te-pierdas-guerra-de-cucharas-y-otras-series-de-tendencia-en-corea-del-sur/
/entretenimiento/2024/09/20/brad-pitt-y-george-clooney-compartieron-como-la-edad-ha-cambiado-sus-perspectivas-sobre-la-vida-y-el-cine/
/entretenimiento/2024/09/20/george-clooney-y-brad-pitt-prometieron-hacerse-cargo-de-matt-damon-en-una-parodia-de-su-nueva-pelicula/
/entretenimiento/2024/09/20/katy-perry-esto-ha-dicho-la-critica-de-su-nuevo-album-143/
/entretenimiento/2024/09/20/katy-perry-ha-encontrado-a-su-critico-personal-en-el-hijo-de-orlando-bloom-el-tiene-un-gran-oido-musical/
/entretenimiento/2024/09/20/steven-adler-confeso-que-la-influencia-de-sus-excompaneros-de-banda-en-

In [9]:
def get_articles_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    main_content = soup.find('article', class_='article').find('div', class_='feed-list-wrapper')
    articles_links = []

    for a_tag in main_content.find_all('a'):
        link = f"https://www.infobae.com{a_tag['href']}"
        if link not in articles_links:
            articles_links.append(link)

    return articles_links

In [11]:
get_articles_links('https://www.infobae.com/wapo/')

['https://www.infobae.com/wapo/2024/09/20/la-economia-china-se-tambalea-y-eso-significa-mas-tensiones-comerciales/',
 'https://www.infobae.com/wapo/2024/09/20/acuerdo-con-microsoft-reabriria-planta-nuclear-de-three-mile-island-para-impulsar-inteligencia-artificial/',
 'https://www.infobae.com/wapo/2024/09/20/es-el-agua-con-gas-igual-de-hidratante-que-el-agua-simple/',
 'https://www.infobae.com/wapo/2024/09/20/las-inundaciones-en-africa-provocan-miseria-y-55-millones-de-personas-ya-padecen-hambre/',
 'https://www.infobae.com/wapo/2024/09/20/johnson-johnson-elevo-su-oferta-por-talco-para-bebes-a-mas-de-usd-8200-millones/',
 'https://www.infobae.com/wapo/2024/09/20/ser-humilde-es-bueno-para-el-aprendizaje-y-para-las-relaciones/',
 'https://www.infobae.com/wapo/2024/09/20/te-duelen-los-ojos-de-tanto-mirar-pantallas-no-te-dejes-enganar-por-las-gafas-con-filtro-de-luz-azul/',
 'https://www.infobae.com/wapo/2024/09/20/los-cientificos-vinculan-nuevamente-el-origen-de-la-pandemia-de-covid-con-l